<img src="Bilder/ost_logo.png" width="240" height="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

# Over- and Underfitting und Regularisierung
[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U03/overfit_and_underfit_SOLUTION-PyTorch.ipynb)


**Overfitting** bedeutet: Unser Modell passt sich den Trainingsdaten *zu sehr* an. Es ist wichtig zu lernen, wie man mit der Überanpassung umgeht. Obwohl es oft möglich ist, eine hohe Genauigkeit im *Trainingsdatensatz* zu erreichen, wollen wir eigentlich Modelle entwickeln, die sich gut auf einen *Testdatensatz* verallgemeinern lassen (oder auf Daten, die sie noch nie gesehen haben).

Das Gegenteil von Overfitting ist **Underfitting**. Underfitting liegt vor, wenn die Trainingsdaten noch verbesserungswürdig sind. Dies kann aus verschiedenen Gründen geschehen: Wenn das Modell nicht leistungsfähig genug ist, überreguliert ist oder einfach nicht lange genug trainiert wurde. Das bedeutet, dass das Netz die relevanten Muster in den Trainingsdaten nicht gelernt hat.

Wenn Sie jedoch zu lange trainieren, wird das Modell anfangen, sich zu sehr anzupassen und Muster aus den Trainingsdaten zu lernen, die sich nicht auf die Testdaten übertragen lassen. Wir müssen also ein Gleichgewicht finden. Es ist nützlich zu wissen, wie man für eine angemessene Anzahl von Epochen trainiert, wie wir weiter unten erläutern werden.

Um eine Überanpassung zu vermeiden, ist die beste Lösung die Verwendung vollständigerer Trainingsdaten. Der Datensatz sollte die gesamte Bandbreite der Eingaben abdecken, die das Modell verarbeiten soll. Zusätzliche Daten sind nur dann sinnvoll, wenn sie neue und interessante Fälle abdecken.
Wir verwenden wie immer die `pytorch_lightning`-API, über die Sie mehr im [PyTorch Lightning Guide](https://pytorch-lightning.readthedocs.io/en/stable/) erfahren können.


## Regularisierung

Ein Modell, das auf umfangreicheren Daten trainiert wurde, wird natürlich besser verallgemeinern. Wenn dies nicht mehr möglich ist, besteht die nächstbeste Lösung darin, Techniken wie die **Regularisierung** anzuwenden. Diese schränken die Menge und Art der Informationen ein, die Ihr Modell speichern kann.  Wenn sich ein Netzwerk nur eine kleine Anzahl von Mustern merken kann, wird es durch den Optimierungsprozess gezwungen, sich auf die auffälligsten Muster zu konzentrieren, die eine bessere Chance haben, gut zu verallgemeinern.

In dieser Übungsaufgabe werden wir verschiedene *gängige Regularisierungstechniken* untersuchen und sie zur Verbesserung eines Klassifizierungsmodells einsetzen.

## Setup

Bevor Sie beginnen, importieren Sie die erforderlichen Pakete:

In [ ]:
# %pip install ipywidgets # für interaktive Widgets und Ladebalken auskommentieren und installieren


In [ ]:
import os

import matplotlib.pyplot as plt  # Zum Erstellen von Diagrammen
import pandas as pd  # Zum Laden und Verarbeiten von CSV-Daten
import pytorch_lightning as pl  # Zur Vereinfachung des Trainingsprozesses
import requests
import torch  # Grundfunktionen und Tensors von PyTorch
import torch.nn as nn  # Neuronale Netzschichten
import torch.optim as optim  # Optimierer von PyTorch
import torchmetrics
from lightning.pytorch.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import (
    EarlyStopping,
)  # Callback, um das Training frühzeitig zu beenden
from pytorch_lightning.loggers import (
    CSVLogger,
)  # Logger für Trainingsmetriken (z. B. in CSV-Dateien)
from torch.utils.data import DataLoader  # Hilfsfunktionen für Datensätze
from torch.utils.data import Dataset, TensorDataset, random_split


## The Higgs Dataset

Das Ziel dieser Übungsserie ist nicht die Teilchenphysik, daher sollten Sie sich nicht mit den Details des Datensatzes beschäftigen. Er enthält 11'000'000 Beispiele, jedes mit 28 Merkmalen und einem binären Klassenlabel.

Informationen zum Datensatz:

Die [Daten](http://mlphysics.ics.uci.edu/data/higgs/) wurden mit Hilfe von Monte-Carlo-Simulationen erstellt. Die ersten 21
Merkmale (Spalten 2-22) sind kinematische Eigenschaften, die von den Teilchendetektoren im Beschleuniger gemessen wurden. Die letzten sieben Merkmale sind Funktionen der ersten 21 Merkmalen; es handelt sich dabei um hochrangige Merkmale, die von Physikern abgeleitet wurden, um zwischen den beiden Klassen zu unterscheiden. 

Es besteht ein grosses Interesse an der Verwendung von Deep Learning Methoden, damit die Physiker solche Merkmale nicht mehr manuell entwickeln müssen. 

Benchmark-Ergebnisse mit Bayes'schen Entscheidungsbäumen aus einem Standard einem Standard-Physikpaket und neuronalen Netzen mit 5 Schichten werden in der ursprünglichen Arbeit vorgestellt. [Baldi, P., P. Sadowski, and D. Whiteson. “Searching for Exotic Particles in High-energy Physics with Deep Learning.” Nature Communications 5 (July 2,
2014)](https://www.nature.com/articles/ncomms5308)



In [ ]:
def download_file_if_not_exists(url, output_dir, filename):
    """
    Lädt eine Datei von der angegebenen URL herunter, wenn sie noch nicht existiert.

    Args:
        url (str): URL der herunterzuladenden Datei.
        output_dir (str): Verzeichnis, in dem die Datei gespeichert wird.
        filename (str): Name der Zieldatei.

    Returns:
        str: Pfad zur heruntergeladenen oder bestehenden Datei.
    """
    # Erstelle das Verzeichnis, falls es nicht existiert
    os.makedirs(output_dir, exist_ok=True)

    # Vollständiger Pfad zur Zieldatei
    output_file = os.path.join(output_dir, filename)

    # Überprüfen, ob die Datei bereits existiert
    if not os.path.exists(output_file):
        print(f"Datei wird heruntergeladen von {url}...")
        # Lade die Datei herunter und speichere sie
        response = requests.get(url, stream=True)
        with open(output_file, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:  # Leere Chunks filtern
                    file.write(chunk)
        print(f"Download abgeschlossen. Datei gespeichert in: {output_file}")
    else:
        print(f"Datei existiert bereits: {output_file}")

    return output_file


# URL der Datei
url = "https://mlphysics.ics.uci.edu/data/higgs/HIGGS.csv.gz"

# Zielverzeichnis und Dateiname
output_dir = "Daten"
filename = "HIGGS.csv.gz"

# Funktion aufrufen
downloaded_file = download_file_if_not_exists(url, output_dir, filename)


## Überanpassung

Der einfachste Weg, eine Überanpassung zu verhindern, ist, mit einem kleinen Modell zu beginnen: Ein Modell mit einer kleinen Anzahl von lernbaren Parametern (die durch die Anzahl der Schichten und die Anzahl der Einheiten pro Schicht bestimmt wird). Beim Deep Learning wird die Anzahl der lernbaren Parameter in einem Modell oft als "Kapazität" des Modells bezeichnet.

Intuitiv betrachtet hat ein Modell mit mehr Parametern eine grössere "Speicherkapazität" und kann daher leicht eine perfekte wörterbuchartige Zuordnung zwischen Trainingsmustern und ihren Zielen erlernen, eine Zuordnung ohne jegliche Generalisierungskraft, die jedoch nutzlos wäre, wenn es Vorhersagen für zuvor nicht gesehene Daten macht.

- Denken Sie immer daran: Deep-Learning-Modelle neigen dazu, sich gut an die Trainingsdaten anzupassen, aber die eigentliche Herausforderung ist die Verallgemeinerung, nicht die Anpassung.

- Wenn das Netzwerk andererseits nur über begrenzte Speicherressourcen verfügt, kann es das Mapping nicht so leicht erlernen. Um seinen Verlust zu minimieren, muss es komprimierte Darstellungen lernen, die eine höhere Vorhersagekraft haben. Wenn Sie Ihr Modell jedoch zu klein machen, wird es Schwierigkeiten haben, sich an die Trainingsdaten anzupassen. Es gibt ein Gleichgewicht zwischen "zu viel Kapazität" und "nicht genug Kapazität".

- Leider gibt es keine magische Formel, um die richtige Grösse oder Architektur Ihres Modells zu bestimmen (in Bezug auf die Anzahl der Schichten oder die richtige Grösse für jede Schicht). Sie müssen mit einer Reihe von verschiedenen Architekturen experimentieren.

- Um eine geeignete Modellgrösse zu finden, beginnen Sie am besten mit relativ wenigen Schichten und Parametern und beginnen dann, die Grösse der Schichten zu erhöhen oder neue Schichten hinzuzufügen, bis Sie eine Verringerung des Validierungsverlustes feststellen.

Beginnen Sie mit einem einfachen Modell, das nur `nn.Linear` als Basis verwendet, erstellen Sie dann grössere Versionen und vergleichen Sie diese.

### Training

Dieses `config_dict` ist ein Python Dictionary, das verschiedene Konfigurationsparameter für das Training eines neuronalen Netzes in PyTorch Lighning enthält. Es ist in drei Hauptabschnitte unterteilt: dataset, model und training.

In [ ]:
config_dict = {
    "dataset": {
        "path": "Daten/HIGGS.csv.gz",  # Pfad zur Datensatzdatei
        "n_train": 10000,  # Anzahl der Trainingsbeispiele
        "n_validation": 1000,  # Anzahl der Validierungsbeispiele
        "batch_size": 50,  # Batch-Größe für Training und Validierung
    },
    "training": {
        "max_epochs": 300,  # Maximale Anzahl der Trainingsepochen
        "early_stopping_patience": 50,  # Geduld für Early Stopping
        "learning_rate": 0.0001,  # Lernrate für den Optimierer
        "use_gpu": True,  # Flag zur Nutzung der GPU für das Training
        "log_dir": "lightning_logs",  # Verzeichnis zum Speichern der Logs
        "log_name": "model_logs",  # Name für die Log-Dateien
    },
}


Die Klasse `HiggDataset` ist eine benutzerdefinierte Datasetklasse. Sie wird verwendet, um den Higgs-Datensatz zu laden und für das Training die Validierung von neuronalen Netzen in PyTorch vorzubereiten.

In [ ]:
# ----------------------------- Dataset Definition -----------------------------
class HiggsDataset(Dataset):
    def __init__(self, data):
        # Die erste Spalte der CSV enthält die Labels (Zielwerte)
        self.labels = (
            torch.tensor(data.iloc[:, 0].values, dtype=torch.float32)
            if not isinstance(data.iloc[:, 0].values, torch.Tensor)
            else data.iloc[:, 0].values
        )
        # Die restlichen Spalten enthalten die Features
        self.features = (
            torch.tensor(data.iloc[:, 1:].values, dtype=torch.float32)
            if not isinstance(data.iloc[:, 1:].values, torch.Tensor)
            else data.iloc[:, 1:].values
        )

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


Die Funktion `create_model` nimmt `input_size`, `hidden_layers` und erstellt die Schichten die im `config_...`-File vorgegeben werden können.

In [ ]:
# ----------------------------- Modellarchitektur -----------------------------
def create_model(input_size, hidden_layers, dropout_rate=0.0, l2_reg=0.0):
    layers = []
    prev_dim = input_size
    for layer_dim in hidden_layers:
        layers.append(nn.Linear(prev_dim, layer_dim))
        layers.append(nn.ELU())
        if dropout_rate > 0:
            layers.append(nn.Dropout(dropout_rate))
        prev_dim = layer_dim
    layers.append(nn.Linear(prev_dim, 1))
    return nn.Sequential(*layers)


Diese Klasse `LightningModel` dient zur Verwaltung von Training, Validierung und Optimierung eines neuronalen Netzwerkes.

In [ ]:
# ----------------------------- LightningModule -----------------------------
class LightningModel(pl.LightningModule):
    def __init__(self, model, lr=0.0001, l2_reg=0.0):
        super(LightningModel, self).__init__()
        self.model = model
        self.lr = lr
        self.l2_reg = l2_reg
        self.training_losses = []  # Speichert den Trainingsverlust pro Epoche
        self.validation_losses = []  # Speichert den Validierungsverlust pro Epoche
        self.training_accuracy = []  # Speichert die Trainingsgenauigkeit pro Epoche
        self.validation_accuracy = []  # Speichert die Validierungsgenauigkeit pro Epoche
        self.accuracy_metric = torchmetrics.Accuracy(task="binary").to(
            self.device
        )  # Genauigkeitsmetrik

    def forward(self, x):
        return self.model(x)  # Vorwärtsdurchlauf

    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.unsqueeze(
            1
        )  # Anpassen der Dimension, damit y mit den Vorhersagen übereinstimmt
        y_hat = self(x)
        loss = nn.BCEWithLogitsLoss()(y_hat, y)  # Berechnung des Verlusts
        # Berechnung der Genauigkeit
        preds = torch.sigmoid(y_hat) > 0.5  # Schwelle bei 0.5
        acc = self.accuracy_metric(preds, y.int())
        self.log(
            "train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )  # Loggen des Trainingsverlusts
        self.log(
            "train_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )  # Loggen der Trainingsgenauigkeit
        return {"loss": loss, "acc": acc}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.unsqueeze(1)  # Anpassen der Dimension
        y_hat = self(x)
        loss = nn.BCEWithLogitsLoss()(y_hat, y)  # Berechnung des Verlusts
        # Berechnung der Genauigkeit
        preds = torch.sigmoid(y_hat) > 0.5
        acc = self.accuracy_metric(preds, y.int())
        self.log(
            "val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )  # Loggen des Validierungsverlusts
        self.log(
            "val_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )  # Loggen der Validierungsgenauigkeit

        return {"loss": loss, "accuracy": acc}

    def on_train_epoch_end(self):
        avg_loss = self.trainer.callback_metrics[
            "train_loss"
        ].item()  # Durchschnittlicher Trainingsverlust
        avg_acc = self.trainer.callback_metrics[
            "train_acc"
        ].item()  # Durchschnittliche Trainingsgenauigkeit
        self.training_losses.append(avg_loss)
        self.training_accuracy.append(avg_acc)

    def on_validation_epoch_end(self):
        avg_loss = self.trainer.callback_metrics[
            "val_loss"
        ].item()  # Durchschnittlicher Validierungsverlust
        avg_acc = self.trainer.callback_metrics[
            "val_acc"
        ].item()  # Durchschnittliche Validierungsgenauigkeit
        self.validation_losses.append(avg_loss)
        self.validation_accuracy.append(avg_acc)

    def configure_optimizers(self):
        optimizer = optim.Adam(
            self.parameters(), lr=self.lr, weight_decay=self.l2_reg
        )  # Optimierer
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode="min", factor=0.1, patience=10
        )  # Lernraten-Scheduler
        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"},
        }


In [ ]:
# ----------------------------- Trainingsfunktion -----------------------------
def train_model(
    model,
    train_loader,
    validate_loader,
    max_epochs=config_dict["training"]["max_epochs"],
    logger=None,
    callbacks=None,
    use_gpu=True,
):
    # Logger festlegen (CSVLogger, falls keiner angegeben ist)
    logger = CSVLogger("logs", name="model_logs") if logger is None else logger

    # Trainer-Instanz von PyTorch Lightning erstellen
    trainer = pl.Trainer(
        max_epochs=max_epochs,  # Maximale Anzahl der Epochen
        logger=logger,  # Logger für Trainingsmetriken
        callbacks=callbacks,  # Callbacks (z.B. EarlyStopping)
        check_val_every_n_epoch=1,  # Validierung nach jeder Epoche
        accelerator="gpu"
        if use_gpu and torch.cuda.is_available()
        else "cpu",  # Beschleuniger (GPU oder CPU)
        devices=[0]
        if use_gpu and torch.cuda.is_available()
        else None,  # Geräte (nur GPU 0, falls verfügbar)
    )

    # Training des Modells
    trainer.fit(model, train_loader, validate_loader)

    # Plotten des Verlaufs der Trainings- und Validierungsverluste
    plt.figure()
    plt.plot(model.training_losses, label="Training Loss")
    plt.plot(model.validation_losses, label="Validation Loss")
    plt.xlabel("Epoche")
    plt.ylabel("Binary Crossentropy Loss")
    plt.legend()
    plt.title("Verlauf der Verluste")
    plt.show()

    return model


In [ ]:
# ----------------------------- Trainingsfunktion mit EarlyStopping -----------------------------
def train_with_early_stopping(
    model,
    train_loader,
    validate_loader,
    max_epochs=config_dict["training"]["max_epochs"],
    use_gpu=True,
):
    # EarlyStopping-Callback erstellen, um das Training zu beenden, wenn sich der Validierungsverlust nicht verbessert
    early_stopping = EarlyStopping(
        monitor="val_loss",  # Überwachen des Validierungsverlusts
        patience=config_dict["training"][
            "early_stopping_patience"
        ],  # Geduld, bevor das Training gestoppt wird
        mode="min",  # Minimierungsmodus, da wir den Verlust minimieren möchten
    )
    # Modell mit EarlyStopping-Callback trainieren
    return train_model(
        model,
        train_loader,
        validate_loader,
        max_epochs=max_epochs,
        callbacks=[early_stopping],
        use_gpu=use_gpu,
    )


In [ ]:
# ----------------------------- Datenvorbereitung -----------------------------
gz_path = config_dict["dataset"]["path"]  # Pfad zur komprimierten Datensatzdatei
data = (
    pd.read_csv(gz_path, compression="gzip", header=None)  # Laden der CSV-Datei
    .sample(frac=1)  # Zufälliges Durchmischen der Daten
    .reset_index(drop=True)  # Zurücksetzen der Indizes
)

N_TRAIN = config_dict["dataset"]["n_train"]  # Anzahl der Trainingsbeispiele
N_VALIDATION = config_dict["dataset"][
    "n_validation"
]  # Anzahl der Validierungsbeispiele
BATCH_SIZE = config_dict["dataset"]["batch_size"]  # Batch-Grösse

data_size = N_TRAIN + N_VALIDATION  # Gesamtanzahl der benötigten Daten
data = data.iloc[:data_size]  # Auswahl der benötigten Daten

# Extrahieren der Features (X) und Labels (y) aus den Daten
X = torch.tensor(data.iloc[:, 1:].values, dtype=torch.float32)
y = torch.tensor(data.iloc[:, 0].values, dtype=torch.float32)

# Erstellen des TensorDatasets und Aufteilen in Trainings- und Validierungsdatensätze
dataset = TensorDataset(X, y)
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(
    dataset, [N_TRAIN, N_VALIDATION], generator=generator
)

# Erstellen der DataLoader für Training und Validierung
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4
)
validate_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4
)


### (c) Winziges Modell

Wir starten mit einem sehr kleinen Modell, das wahrscheinlich nicht an Überanpassung leidet.

config file vor jedem training und gut am Anfang beschreiben, und alle helper Funktionen

In [ ]:
config_tiny = {
    "model": {
        "input_size": 28,  # Eingangsgröße des Modells
        "tiny": {
            "hidden_layers": [16],
            "dropout_rate": 0.0,
            "l2_reg": 0.0,
        },  # Konfiguration für Tiny-Modell
    },
}


In [ ]:
print("Training Basis Tiny Model...")
# printe die Konfiguration des Modells
print(config_tiny["model"]["tiny"])
tiny_model = create_model(
    config_tiny["model"]["input_size"], **config_tiny["model"]["tiny"]
)
tiny_lightning_model = LightningModel(tiny_model)
tiny_trained = train_with_early_stopping(
    tiny_lightning_model, train_loader, validate_loader
)


### (d) Kleines Modell

Um zu sehen, ob Sie die Leistung des kleinen Modells übertreffen können, trainieren Sie nach und nach einige grössere Modelle.

Versuchen Sie es mit zwei versteckten Schichten mit je 16 Einheiten:

In [ ]:
config_small = {
    "model": {
        "input_size": 28,  # Eingangsgröße des Modells
        "small": {
            "hidden_layers": [16, 16],
            "dropout_rate": 0.0,
            "l2_reg": 0.0,
        },  # Konfiguration für Small-Modell
    },
}


In [ ]:
print("Training Small Model (Basis, [16,16])...")
# printe die Konfiguration des Modells
print(config_small["model"]["small"])
small_model = create_model(
    config_small["model"]["input_size"], **config_small["model"]["small"]
)
small_lightning_model = LightningModel(small_model)
small_trained = train_with_early_stopping(
    small_lightning_model, train_loader, validate_loader
)


### (e) Mittelgrosses Modell

Testen Sie nun ein Modell mit drei versteckten Schichten (hidden layers) mit je 64 Einheiten:

In [ ]:
config_medium = {
    "model": {
        "input_size": 28,  # Eingangsgröße des Modells
        "medium": {
            "hidden_layers": [64, 64, 64],
            "dropout_rate": 0.0,
            "l2_reg": 0.0,
        },  # Konfiguration für Medium-Modell
    },
}


In [ ]:
print("Training Medium Model (Basis, [64,64,64])...")
# printe die Konfiguration des Modells
print(config_medium["model"]["medium"])
medium_model = create_model(
    config_medium["model"]["input_size"], **config_medium["model"]["medium"]
)
medium_lightning_model = LightningModel(medium_model)
medium_trained = train_with_early_stopping(
    medium_lightning_model, train_loader, validate_loader
)


### (f) grosses Modell

Zu Übungszwecken können Sie ein noch grösseres Modell erstellen und sehen, wie schnell es anfängt, sich übermässig anzupassen.  Als Nächstes fügen wir zu diesem Benchmark ein Netzwerk hinzu, das viel mehr Kapazität hat, weit mehr als das Problem rechtfertigen würde:

In [ ]:
config_large = {
    "model": {
        "input_size": 28,  # Eingangsgröße des Modells
        "large": {
            "hidden_layers": [512, 512, 512, 512],
            "dropout_rate": 0.0,
            "l2_reg": 0.0,
        },  # Konfiguration für Large-Modell
    },
}


In [ ]:
print("Training Large Model (Basis, [512,512,512,512])...")
# printe die Konfiguration des Modells
print(config_large["model"]["large"])
large_model = create_model(
    config_large["model"]["input_size"], **config_large["model"]["large"]
)
large_lightning_model = LightningModel(large_model)
large_trained = train_with_early_stopping(
    large_lightning_model, train_loader, validate_loader
)


### (g) Interpretation

Die Erstellung eines grösseren Modells verleiht ihm zwar mehr Leistung, aber wenn diese Leistung nicht irgendwie eingeschränkt wird, kann es leicht zu einer Überanpassung an den Trainingssatz kommen.

In diesem Beispiel gelingt es typischerweise nur dem "kleinen" Modell, eine Überanpassung ganz zu vermeiden, und jedes der größeren Modelle passt die Daten schneller über. Für das "große" Modell ist dies so gravierend, dass man die Darstellung auf eine logarithmische Skala umstellen muss, um wirklich zu sehen, was passiert.

Dies wird deutlich, wenn Sie die Validierungsmetriken aufzeichnen und mit den Trainingsmetriken vergleichen.

* Es ist normal, dass es einen kleinen Unterschied gibt.
* Wenn sich beide Metriken in dieselbe Richtung bewegen, ist alles in Ordnung.
* Wenn die Validierungskennzahl zu stagnieren beginnt, während sich die Trainingskennzahl weiter verbessert, sind Sie wahrscheinlich nahe an einer Überanpassung.
* Wenn sich die Validierungsmetrik in die falsche Richtung bewegt, ist das Modell eindeutig überangepasst.

Die durchgezogenen Linien zeigen den Trainingsverlust, die gestrichelten Linien den Validierungsverlust (zur Erinnerung: ein geringerer Validierungsverlust bedeutet ein besseres Modell).

Note: All the above training runs used the `callbacks.EarlyStopping` to end the training once it was clear the model was not making progress.

## Strategien zur Regularisierung

Bevor Sie sich mit dem Inhalt dieses Abschnitts befassen, kopieren Sie die Trainingsprotokolle des obigen Modells "Tiny", um sie als Vergleichsgrundlage zu verwenden.

### (h) Gewichtsregularisierung (weight regularization)


Vielleicht kennen Sie das Prinzip von **Occams Rasiermesser**: Wenn es zwei Erklärungen für etwas gibt, ist die Erklärung, die am wahrscheinlichsten richtig ist, die "einfachste", diejenige, die die wenigsten Annahmen enthält. Dies gilt auch für die Modelle, die von neuronalen Netzen gelernt werden: Bei bestimmten Trainingsdaten und einer Netzarchitektur gibt es mehrere Sätze von Gewichtungswerten (mehrere Modelle), die die Daten erklären könnten, und bei einfacheren Modellen ist die Wahrscheinlichkeit einer Überanpassung geringer als bei komplexen Modellen.

Ein "einfaches Modell" ist in diesem Zusammenhang ein Modell, bei dem die Verteilung der Parameterwerte eine geringere Entropie aufweist (oder ein Modell mit insgesamt weniger Parametern, wie wir im obigen Abschnitt gesehen haben). Eine gängige Methode zur Abschwächung der Überanpassung besteht daher darin, die Komplexität eines Netzes einzuschränken, indem seine Gewichte gezwungen werden, nur kleine Werte anzunehmen, wodurch die Verteilung der Gewichtswerte "regelmässiger" wird. Dies wird als "Gewichtsregulierung" bezeichnet und erfolgt durch Hinzufügen von Kosten zur Verlustfunktion des Netzes, die mit großen Gewichten verbunden sind. Diese Kosten gibt es in zwei Varianten:

* [L1-Regularisierung](https://developers.google.com/machine-learning/glossary/#L1_regularization), bei der die hinzugefügten Kosten proportional zum absoluten Wert der Gewichtskoeffizienten sind (d. h. zur so genannten "L1-Norm" der Gewichte).

* [L2-Regularisierung](https://developers.google.com/machine-learning/glossary/#L2_regularization), bei der die zusätzlichen Kosten proportional zum Quadrat des Wertes der Gewichtungskoeffizienten sind (d.h. zu dem, was man die quadrierte "L2-Norm" der Gewichte nennt). Die L2-Regularisierung wird im Zusammenhang mit neuronalen Netzen auch als Gewichtsabnahme bezeichnet. Lassen Sie sich durch die unterschiedliche Bezeichnung nicht verwirren: Gewichtsabnahme ist mathematisch gesehen genau dasselbe wie L2-Regularisierung.

Die L1-Regularisierung verschiebt einige Gewichte genau gegen Null, was ein spärliches Modell fördert. Die L2-Regularisierung bestraft die Parameter der Gewichte, ohne sie spärlich zu machen, da die Strafe bei kleinen Gewichten gegen Null geht - ein Grund, warum L2 häufiger verwendet wird.


`l2(0.001)` bedeutet, dass jeder Koeffizient in der Gewichtsmatrix der Schicht `0.001 * weight_coefficient_value**2` zum gesamten **Verlust** des Netzes beiträgt.

Deshalb überwachen wir die `binary_crossentropy` direkt, weil sie diese Regularisierungskomponente nicht enthält.



Es gibt einen zweiten Ansatz, bei dem der Optimierer nur auf den Rohverlust angewandt wird und dann während der Anwendung des berechneten Schritts auch ein gewisser Gewichtsabbau erfolgt. Dieses "Decoupled Weight Decay" findet sich in Optimierern wie `torch.optim.AdamW`. (`torch.optim.Adam`-> gekoppeltes Weight Decay)

In [ ]:
config_L2 = {
    "model": {
        "input_size": 28,  # Eingangsgröße des Modells
        "large_l2": {
            "hidden_layers": [512, 512, 512, 512],
            "dropout_rate": 0.0,
            "l2_reg": 0.001,
        },  # Tiny-Modell mit L2-Regularisierung
    },
}


In [ ]:
print("Training Large Model mit L2-Regularisierung...")
# printe die Konfiguration des Modells
print(config_L2["model"]["large_l2"])
large_l2_model = create_model(
    config_L2["model"]["input_size"], **config_L2["model"]["large_l2"]
)
large_l2_lightning_model = LightningModel(
    large_l2_model, l2_reg=config_L2["model"]["large_l2"]["l2_reg"]
)
large_l2_trained = train_with_early_stopping(
    large_l2_lightning_model, train_loader, validate_loader
)


### (i) Dropout hinzufügen

Dropout ist eine der effektivsten und am häufigsten verwendeten Regularisierungstechniken für neuronale Netze, die von Hinton und seinen Studenten an der Universität von Toronto entwickelt wurde.

Die intuitive Erklärung für Dropout ist, dass sich einzelne Knoten im Netz nicht auf die Ausgaben der anderen verlassen können, sondern dass jeder Knoten für sich selbst nützliche Merkmale ausgeben muss.

Dropout, angewandt auf eine Schicht, besteht darin, dass während des Trainings eine Anzahl von Ausgangsmerkmalen der Schicht zufällig "weggelassen" (d. h. auf Null gesetzt) wird. Nehmen wir an, eine gegebene Schicht hätte normalerweise einen Vektor `[0.2, 0.5, 1.3, 0.8, 1.1]` für eine gegebene Eingabeprobe während des Trainings geliefert; nach Anwendung von Dropout wird dieser Vektor einige zufällig verteilte Nulleinträge haben, z. B. `[0, 0.5, 1.3, 0, 1.1]`.

Die "Dropout-Rate" ist der Anteil der Merkmale, die mit Nullen versehen werden; sie wird normalerweise zwischen `0.2` und `0.5` festgelegt. Zur Testzeit werden keine Einheiten herausgenommen, stattdessen werden die Ausgabewerte der Schicht um den Faktor der Dropout-Rate herunterskaliert, um die Tatsache auszugleichen, dass mehr Einheiten aktiv sind als zur Trainingszeit.

In `PyTorch Lightning` können Sie Dropouts in einem Netzwerk über die `nn.Dropout`-Schicht einführen, die auf die Ausgabe der Schicht direkt davor angewendet wird.

Fügen wir Dropout-Schichten in unser Netzwerk ein, um zu sehen, wie gut sie das Overfitting reduzieren:

In [ ]:
config_dropout = {
    "model": {
        "input_size": 28,  # Eingangsgröße des Modells
        "large_dropout": {
            "hidden_layers": [512, 512, 512, 512],
            "dropout_rate": 0.5,
            "l2_reg": 0.0,
        },  # Tiny-Modell mit Dropout
    },
}


In [ ]:
print("Training Large Model mit Dropout...")
# printe die Konfiguration des Modells
print(config_dropout["model"]["large_dropout"])
large_dropout_model = create_model(
    config_dropout["model"]["input_size"], **config_dropout["model"]["large_dropout"]
)
large_dropout_lightning_model = LightningModel(large_dropout_model)
large_dropout_trained = train_with_early_stopping(
    large_dropout_lightning_model, train_loader, validate_loader
)


### (j) L2-Regularisierung und Dropout kombiniert

Versuchen Sie als Nächstes beide zusammen, um zu sehen, ob das besser funktioniert.

In [ ]:
config_L2_dropout = {
    "model": {
        "input_size": 28,  # Eingangsgröße des Modells
        "large_l2_dropout": {
            "hidden_layers": [512, 512, 512, 512],
            "dropout_rate": 0.5,
            "l2_reg": 0.001,
        },  # Tiny-Modell mit Dropout und L2-Regularisierung
    },
}


In [ ]:
print("Training Large Model mit Dropout und L2-Regularisierung...")
# printe die Konfiguration des Modells
print(config_L2_dropout["model"]["large_l2_dropout"])
large_l2_dropout_model = create_model(
    config_L2_dropout["model"]["input_size"],
    **config_L2_dropout["model"]["large_l2_dropout"],
)
large_l2_dropout_lightning_model = LightningModel(
    large_l2_dropout_model,
    l2_reg=config_L2_dropout["model"]["large_l2_dropout"]["l2_reg"],
)
large_l2_dropout_trained = train_with_early_stopping(
    large_l2_dropout_lightning_model, train_loader, validate_loader
)


In [ ]:
def plot_model_comparisons(models, metrics, titles, xlabel="Epoche"):
    """
    Erstellt Vergleichsplots für verschiedene Modelle und Metriken als Subplots.

    Args:
        models (dict): Dictionary mit Modellnamen als Keys und einem weiteren Dictionary mit Metriken als Werten.
                       Beispiel:
                       {
                           "Tiny Basis [16]": tiny_trained,
                           "Tiny + Dropout": tiny_dropout_trained
                       }
        metrics (list): Liste der Metriken als Strings, z.B. ["training_losses", "validation_losses"]
        titles (list): Liste der Titel für die Plots in der gleichen Reihenfolge wie die Metriken.
        xlabel (str): Bezeichnung der x-Achse (Standard: "Epoche").
    """
    num_metrics = len(metrics)
    fig, axes = plt.subplots(num_metrics, 1, figsize=(10, 6 * num_metrics))

    for ax, metric, title in zip(axes, metrics, titles):
        for model_name, model_data in models.items():
            ax.plot(getattr(model_data, metric), label=model_name)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(metric.replace("_", " ").capitalize())
        ax.set_title(title)
        ax.legend()
        ax.grid(True)

    plt.tight_layout()
    plt.show()


# ----------------------------- Anwendung der Funktion -----------------------------

# 1. Vergleich der Tiny-Modellvarianten
tiny_models = {
    "Tiny Basis [16]": tiny_trained,
    "large + Dropout": large_dropout_trained,
    "large + L2": large_l2_trained,
    "large + Dropout und L2": large_l2_dropout_trained,
}

plot_model_comparisons(
    tiny_models,
    metrics=[
        "training_losses",
        "validation_losses",
        "training_accuracy",
        "validation_accuracy",
    ],
    titles=[
        "Vergleich der Trainingsverluste",
        "Vergleich der Validierungsverluste",
        "Vergleich der Trainings-Accuracy",
        "Vergleich der Validierungs-Accuracy",
    ],
)

# 2. Vergleich der Basis-Modelle
base_models = {
    "Tiny Basis [16]": tiny_trained,
    "Small [16,16]": small_trained,
    "Medium [64,64,64]": medium_trained,
    "Large [512,512,512,512]": large_trained,
}

plot_model_comparisons(
    base_models,
    metrics=["training_losses", "validation_losses"],
    titles=[
        "Vergleich der Trainingsverluste: Basis-Modelle",
        "Vergleich der Validierungsverluste: Basis-Modelle",
    ],
)


# Schlussfolgerungen

Zusammenfassend lässt sich sagen, dass es die gängigsten Methoden gibt, um eine Überanpassung in neuronalen Netzen zu verhindern:

* Mehr Trainingsdaten erhalten.
* Verringern Sie die Kapazität des Netzes.
* Hinzufügen einer Gewichtsregulierung.
* Dropout hinzufügen.

Zwei wichtige Ansätze, die in diesem Leitfaden nicht behandelt werden, sind:

* Datenerweiterung
* Batch-Normalisierung

Denken Sie daran, dass jede Methode für sich genommen hilfreich sein kann, aber oft kann eine Kombination der Methoden noch effektiver sein.

# Zusammenfassung Over- und Underfitting erkennen
| Kategorie | Underfitting | Overfitting |
|-----------|--------------|-------------|
| **Symptome** | - Hoher Trainingsfehler<br>- Hoher Validierungsfehler<br>- Trainings- und Validierungskurven liegen nahe beieinander | - Sehr niedriger Trainingsfehler<br>- Hoher Validierungsfehler<br>- Grosse Lücke zwischen Trainings- und Validierungskurve |
| **Ursachen** | - Modell zu einfach<br>- Zu starke Regularisierung<br>- Zu kurze Trainingszeit | - Modell zu komplex<br>- Zu langes Training<br>- Zu wenig Trainingsdaten |
| **Lösungen** | - Komplexeres Modell verwenden<br>- Regularisierung reduzieren<br>- Mehr Trainingszeit (Epochen) | - Regularisierung hinzufügen/erhöhen (L1/L2)<br>- Dropout verwenden<br>- Data Augmentation einsetzen<br>- Earlystopping anwenden |



